In [1]:
'''Import libraries'''
import pandas as pd
from scipy.stats import norm
import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
'''Load csv file'''
data = pd.read_csv("mbti_1.csv")


'''find number of rows and columns of data'''
print("the number of rows and columns of data",data.shape,'\n')

'''View information about the dataset'''
data.info()


'''observe posts content'''
#print(data.iloc[0,1])

'''Check for missing values'''
print()
print("Check for missing values", data.isna().sum())


the number of rows and columns of data (8675, 2) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 67.8+ KB

Check for missing values type     0
posts    0
dtype: int64


In [3]:
data.describe()

,type,posts
count,8675,8675
unique,16,8675
top,INFP,'Always striving for more interesting possibil...
freq,1832,1


In [4]:
'''Find the types of personality'''
data['type'].unique()

array(['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'],
      dtype=object)

In [5]:
'''Find the count of each personality type'''
category_count = data['type'].value_counts()
category_count

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

# Data preprocessing 

In [7]:
dataframe = data.copy()
dataframe.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [11]:
def prepocessing_regex(text):
    text=re.sub(r'\|\|\|', r'', text) # removing |||
    text=re.sub(r'http\S+', r'', text) # removing http.
    text=text.replace(".", "").replace(",","").replace('+', '').replace(":","").replace('"','').replace("'",'')
    text=text.replace(";",'').replace('?',"").replace('_____','').replace('-','').replace('/','').replace("@","")
    text=text.replace("!",'')
    text=re.sub(r'[()]', '', text) # removing '(' or ')'
    text=re.sub(r'[<>]', '', text) # removing '<' or '>'
    text=text.replace("*","").lower()
    text=re.sub("[^a-zA-Z]"," ",text) #keep only words
    prepocessed_text=text
    
    stop_words = (stopwords.words('english')) 
                                               # mbti types will be removed in the posts for training and test 
    stop_words.extend(('infj', 'entp', 'intp', 'intj', 'entj', 'enfj', 'infp', 
                   'enfp', 'isfp', 'istp', 'isfj', 'istj', 'estp', 'esfp', 'estj', 'esfj',
                   'INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP','ISFP', 
                   'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'))
    stop_words=set(stop_words)
    word_tokens=word_tokenize(prepocessed_text)
    word_tokenized_preprocessed=[]

    for token in word_tokens:
        if token not in stop_words:
            word_tokenized_preprocessed.append(token)
            
    s=word_tokenized_preprocessed
    converting_stopwords = ' '.join(map(str, s))   # converting list to string
    stop_words=[converting_stopwords]
    
    return stop_words

In [12]:
def Lemmantization(prepocessed_words):
    n=WordNetLemmatizer()
    words_for_lemman=prepocessed_words
    lemmantized_word=[n.lemmatize(w) for w in words_for_lemman]           
    to_str=lemmantized_word
    lemmantized_word=' '.join(map(str, to_str))   # converting list to string
    
    return lemmantized_word

In [14]:
def snow_stemmer(prepocessed_words):
    snow_stemmer = SnowballStemmer(language='english')
    words_for_snow_stemmer= prepocessed_words
    snowballstemmed_words=[snow_stemmer.stem(w) for w in words_for_snow_stemmer]
    return snowballstemmed_words
    
def stemmer_counter(text):
    out=CountVectorizer(text)    
    return out

def stemmer_tf_idf(text):
    out=TfidfVectorizer(text)   
    return out




In [15]:
def lem_counter(text):
    out=CountVectorizer(text)    
    return out

def lem_tf_idf(text):
    out=TfidfVectorizer(text)   
    return out


In [70]:
# texts are preprocessed and stop words are removed
dataframe['processed'] = dataframe['posts'].apply(prepocessing_regex)

dataframe.head()

,type,posts,processed,lemmantized,snow_stemmer,lem_counter,lem_tf_idf,stemmer_counter,stemmer_tfidf
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,[moments sportscenter top ten plays prankswhat...,moments sportscenter top ten plays prankswhat ...,[moments sportscenter top ten plays prankswhat...,CountVectorizer(input='moments sportscenter to...,TfidfVectorizer(input='moments sportscenter to...,CountVectorizer(input=['moments sportscenter t...,TfidfVectorizer(input=['moments sportscenter t...
1,ENTP,'I'm finding the lack of me in these posts ver...,[im finding lack posts alarmingsex boring posi...,im finding lack posts alarmingsex boring posit...,[im finding lack posts alarmingsex boring posi...,CountVectorizer(input='im finding lack posts a...,TfidfVectorizer(input='im finding lack posts a...,CountVectorizer(input=['im finding lack posts ...,TfidfVectorizer(input=['im finding lack posts ...
2,INTP,'Good one _____ https://www.youtube.com/wat...,[good one course say know thats blessing curse...,good one course say know thats blessing cursed...,[good one course say know thats blessing curse...,CountVectorizer(input='good one course say kno...,TfidfVectorizer(input='good one course say kno...,CountVectorizer(input=['good one course say kn...,TfidfVectorizer(input=['good one course say kn...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",[dear enjoyed conversation day esoteric gabbin...,dear enjoyed conversation day esoteric gabbing...,[dear enjoyed conversation day esoteric gabbin...,CountVectorizer(input='dear enjoyed conversati...,TfidfVectorizer(input='dear enjoyed conversati...,CountVectorizer(input=['dear enjoyed conversat...,TfidfVectorizer(input=['dear enjoyed conversat...
4,ENTJ,'You're fired.|||That's another silly misconce...,[youre firedthats another silly misconception ...,youre firedthats another silly misconception a...,[youre firedthats another silly misconception ...,CountVectorizer(input='youre firedthats anothe...,TfidfVectorizer(input='youre firedthats anothe...,CountVectorizer(input=['youre firedthats anoth...,TfidfVectorizer(input=['youre firedthats anoth...


In [17]:
#applying lemmatization
dataframe['lemmantized'] = dataframe['processed'].apply(Lemmantization)
dataframe.head()



,type,posts,processed,lemmantized
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,[moments sportscenter top ten plays prankswhat...,moments sportscenter top ten plays prankswhat ...
1,ENTP,'I'm finding the lack of me in these posts ver...,[im finding lack posts alarmingsex boring posi...,im finding lack posts alarmingsex boring posit...
2,INTP,'Good one _____ https://www.youtube.com/wat...,[good one course say know thats blessing curse...,good one course say know thats blessing cursed...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",[dear enjoyed conversation day esoteric gabbin...,dear enjoyed conversation day esoteric gabbing...
4,ENTJ,'You're fired.|||That's another silly misconce...,[youre firedthats another silly misconception ...,youre firedthats another silly misconception a...


In [20]:
#applying stemmer
dataframe['snow_stemmer'] = dataframe['processed'].apply(snow_stemmer)


In [21]:
dataframe.head()

,type,posts,processed,lemmantized,snow_stemmer
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,[moments sportscenter top ten plays prankswhat...,moments sportscenter top ten plays prankswhat ...,[moments sportscenter top ten plays prankswhat...
1,ENTP,'I'm finding the lack of me in these posts ver...,[im finding lack posts alarmingsex boring posi...,im finding lack posts alarmingsex boring posit...,[im finding lack posts alarmingsex boring posi...
2,INTP,'Good one _____ https://www.youtube.com/wat...,[good one course say know thats blessing curse...,good one course say know thats blessing cursed...,[good one course say know thats blessing curse...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",[dear enjoyed conversation day esoteric gabbin...,dear enjoyed conversation day esoteric gabbing...,[dear enjoyed conversation day esoteric gabbin...
4,ENTJ,'You're fired.|||That's another silly misconce...,[youre firedthats another silly misconception ...,youre firedthats another silly misconception a...,[youre firedthats another silly misconception ...


In [36]:
# CountVectorization result using stemmer

def Print_stemmer_counter(text): 
    vectorizer = CountVectorizer()
    vectorizer.fit(text)
    print(vectorizer.vocabulary_)# encode document
    vector = vectorizer.transform(text)
    # summarize encoded vector
    print(vector.shape)
    print(vector.toarray())
    

Print_stemmer_counter(dataframe['processed'][0])

{'moments': 134, 'sportscenter': 203, 'top': 227, 'ten': 212, 'plays': 159, 'prankswhat': 163, 'lifechanging': 117, 'experience': 66, 'life': 116, 'repeat': 173, 'todaymay': 224, 'perc': 154, 'immerse': 99, 'youthe': 258, 'last': 108, 'thing': 213, 'friend': 75, 'posted': 162, 'facebook': 67, 'committing': 31, 'suicide': 208, 'next': 142, 'day': 44, 'rest': 175, 'peace': 151, 'sorry': 200, 'hear': 90, 'distress': 53, 'natural': 140, 'relationship': 172, 'perfection': 155, 'time': 222, 'every': 62, 'moment': 133, 'existence': 65, 'try': 229, 'figure': 72, 'hard': 86, 'times': 223, 'growth': 83, 'welcome': 242, 'stuff': 206, 'game': 77, 'set': 187, 'matchprozac': 125, 'wellbrutin': 243, 'least': 113, 'thirty': 217, 'minutes': 131, 'moving': 136, 'legs': 115, 'dont': 54, 'mean': 127, 'sitting': 193, 'desk': 49, 'chair': 25, 'weed': 241, 'moderation': 132, 'maybe': 126, 'edibles': 59, 'healthier': 89, 'alternativebasically': 2, 'come': 30, 'three': 220, 'items': 103, 'youve': 259, 'determi

In [52]:
#consider clean data

df = dataframe[['type', 'lemmantized']]

In [53]:
dataframe['lem_counter'] = dataframe['lemmantized'].apply(lem_counter)
dataframe.head()

,type,posts,processed,lemmantized,snow_stemmer,lem_counter,lem_tf_idf,stemmer_counter
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,[moments sportscenter top ten plays prankswhat...,moments sportscenter top ten plays prankswhat ...,[moments sportscenter top ten plays prankswhat...,CountVectorizer(input='moments sportscenter to...,TfidfVectorizer(input='moments sportscenter to...,CountVectorizer(input=['moments sportscenter t...
1,ENTP,'I'm finding the lack of me in these posts ver...,[im finding lack posts alarmingsex boring posi...,im finding lack posts alarmingsex boring posit...,[im finding lack posts alarmingsex boring posi...,CountVectorizer(input='im finding lack posts a...,TfidfVectorizer(input='im finding lack posts a...,CountVectorizer(input=['im finding lack posts ...
2,INTP,'Good one _____ https://www.youtube.com/wat...,[good one course say know thats blessing curse...,good one course say know thats blessing cursed...,[good one course say know thats blessing curse...,CountVectorizer(input='good one course say kno...,TfidfVectorizer(input='good one course say kno...,CountVectorizer(input=['good one course say kn...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",[dear enjoyed conversation day esoteric gabbin...,dear enjoyed conversation day esoteric gabbing...,[dear enjoyed conversation day esoteric gabbin...,CountVectorizer(input='dear enjoyed conversati...,TfidfVectorizer(input='dear enjoyed conversati...,CountVectorizer(input=['dear enjoyed conversat...
4,ENTJ,'You're fired.|||That's another silly misconce...,[youre firedthats another silly misconception ...,youre firedthats another silly misconception a...,[youre firedthats another silly misconception ...,CountVectorizer(input='youre firedthats anothe...,TfidfVectorizer(input='youre firedthats anothe...,CountVectorizer(input=['youre firedthats anoth...


In [54]:

dataframe['lem_tf_idf'] = dataframe['lemmantized'].apply(lem_tf_idf)


In [55]:

dataframe['stemmer_counter'] = dataframe['snow_stemmer'].apply(stemmer_counter)


In [56]:
dataframe['stemmer_tfidf'] = dataframe['snow_stemmer'].apply(stemmer_tf_idf)
dataframe.head()

,type,posts,processed,lemmantized,snow_stemmer,lem_counter,lem_tf_idf,stemmer_counter,stemmer_tfidf
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,[moments sportscenter top ten plays prankswhat...,moments sportscenter top ten plays prankswhat ...,[moments sportscenter top ten plays prankswhat...,CountVectorizer(input='moments sportscenter to...,TfidfVectorizer(input='moments sportscenter to...,CountVectorizer(input=['moments sportscenter t...,TfidfVectorizer(input=['moments sportscenter t...
1,ENTP,'I'm finding the lack of me in these posts ver...,[im finding lack posts alarmingsex boring posi...,im finding lack posts alarmingsex boring posit...,[im finding lack posts alarmingsex boring posi...,CountVectorizer(input='im finding lack posts a...,TfidfVectorizer(input='im finding lack posts a...,CountVectorizer(input=['im finding lack posts ...,TfidfVectorizer(input=['im finding lack posts ...
2,INTP,'Good one _____ https://www.youtube.com/wat...,[good one course say know thats blessing curse...,good one course say know thats blessing cursed...,[good one course say know thats blessing curse...,CountVectorizer(input='good one course say kno...,TfidfVectorizer(input='good one course say kno...,CountVectorizer(input=['good one course say kn...,TfidfVectorizer(input=['good one course say kn...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",[dear enjoyed conversation day esoteric gabbin...,dear enjoyed conversation day esoteric gabbing...,[dear enjoyed conversation day esoteric gabbin...,CountVectorizer(input='dear enjoyed conversati...,TfidfVectorizer(input='dear enjoyed conversati...,CountVectorizer(input=['dear enjoyed conversat...,TfidfVectorizer(input=['dear enjoyed conversat...
4,ENTJ,'You're fired.|||That's another silly misconce...,[youre firedthats another silly misconception ...,youre firedthats another silly misconception a...,[youre firedthats another silly misconception ...,CountVectorizer(input='youre firedthats anothe...,TfidfVectorizer(input='youre firedthats anothe...,CountVectorizer(input=['youre firedthats anoth...,TfidfVectorizer(input=['youre firedthats anoth...


In [ ]:
#  X will be chosen by us from 4 cases.
#X= dataframe['stemmer_tfidf'] or dataframe['stemmer_counter'] or dataframe['lem_counter'] or dataframe['lem_tf_idf']

# y will be label 
#y=dataframe['type'] 
#y

In [60]:
#consider clean data

df = dataframe[['type', 'lemmantized']]

df.head()

,type,lemmantized
0,INFJ,moments sportscenter top ten plays prankswhat ...
1,ENTP,im finding lack posts alarmingsex boring posit...
2,INTP,good one course say know thats blessing cursed...
3,INTJ,dear enjoyed conversation day esoteric gabbing...
4,ENTJ,youre firedthats another silly misconception a...


In [61]:
#after X are confirmed, we will find optimized parameter of chosen X according to the results 

In [62]:
vect=CountVectorizer(
                             #analyzer = 'word',
                             tokenizer=None, 
                             preprocessor = None, 
                             stop_words = None, 
                             #ngram_range=(1, 3), 
                             max_df=.8,
                             min_df=3,                        
                             max_features = 2000
                            )

In [63]:
X =  vect.fit_transform(df["lemmantized"]) 

In [64]:
le = LabelEncoder()
y = le.fit_transform(df["type"])

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)

In [71]:
X_train, X_test, y_train, y_test

(<6072x2000 sparse matrix of type '<class 'numpy.int64'>'
 	with 1582858 stored elements in Compressed Sparse Row format>,
 <2603x2000 sparse matrix of type '<class 'numpy.int64'>'
 	with 674954 stored elements in Compressed Sparse Row format>,
 array([ 9,  8,  9, ...,  2, 11,  4]),
 array([ 2, 15, 11, ...,  3,  3,  9]))